## 3.3 线性回归的简洁实现

### 3.1 生成数据集

In [1]:
import numpy as np
import torch
from torch.utils import data
from d2l import torch as d2l

true_w = torch.tensor([2, -3.4])
true_b = 4.2
features, labels = d2l.synthetic_data(true_w, true_b, 1000)

### 3.3.2 读取数据集

调用现有API来读取数据，也就是`torch.util.data.DataLoader`

需要注意的是下面出现的`*data_array`不是指针的意思，而是**解包**。也就是将`data_array`中的每个元素单独的传递给函数，而不是作为一个整体来传递。`*`适用的对象可以是`tuple`或`list`。

对于下面这个示例而言，下面两种写法完全等价：
- `dataset = data.TensorDataset(*data_arrays)`
- `dataset = data.TensorDataset(features, labels)`

In [2]:
def load_array(data_arrays, batch_size, is_train=True):  #@save
    """构造一个PyTorch数据迭代器"""
    dataset = data.TensorDataset(*data_arrays)
    return data.DataLoader(dataset, batch_size, shuffle=is_train)

batch_size = 10
data_iter = load_array((features, labels), batch_size)

In [3]:
next(iter(data_iter))

[tensor([[-0.3618,  0.1690],
         [-0.3609, -0.2891],
         [-0.8798,  0.4808],
         [-0.7710, -0.7033],
         [-0.6539,  0.2537],
         [ 1.4595,  0.4163],
         [ 0.7944, -0.2188],
         [ 1.3803, -0.3435],
         [-0.2776,  0.4252],
         [ 0.3081, -0.3946]]),
 tensor([[2.9002],
         [4.4446],
         [0.8169],
         [5.0568],
         [2.0398],
         [5.6956],
         [6.5411],
         [8.1294],
         [2.2058],
         [6.1594]])]

### 3.3.3 定义模型

使用torch.nn模块定义模型，其中需要构造一个实例，便使用`Sequential`构造实例

In [4]:
from torch import nn

net = nn.Sequential(nn.Linear(2, 1))

### 3.3.4 初始化模型参数

使用预定义的方法初始化模型参数

`net[0]`的作用是选中第一个图层

In [7]:
net[0].weight.data.normal_(0, 0.01)
net[0].bias.data.fill_(0)

tensor([0.])

### 3.3.5 定义损失函数

使用平方$L_2$范数

In [8]:
loss = nn.MSELoss()

### 3.3.6 定义优化算法

In [11]:
trainer = torch.optim.SGD(net.parameters(), lr=0.03)

### 3.3.7 训练

In [12]:
num_epochs = 3
for epoch in range(num_epochs):
    for X, y in data_iter:
        # 进行预测，并计算损失，还不涉及梯度的计算
        l = loss(net(X), y)
        # 在反向传播之前，使用优化器对象将梯度清零
        trainer.zero_grad()
        l.backward()
        trainer.step()
    l = loss(net(features), labels)
    print(f'epoch {epoch + 1}, loss {l:f}')

epoch 1, loss 0.000208
epoch 2, loss 0.000103
epoch 3, loss 0.000101


- `w.shape=torch.Size([1, 2])`
- `true_w.shape=torch.Size([2])`

两者不同，故需要进行`reshape`。

In [14]:
w = net[0].weight.data
print('w的估计误差：', true_w - w.reshape(true_w.shape))
b = net[0].bias.data
print('b的估计误差：', true_b - b)

torch.Size([2]) torch.Size([1, 2])
w的估计误差： tensor([-0.0011,  0.0003])
b的估计误差： tensor([-0.0001])


### 3.3.8 小结

- 我们可以使用PyTorch的高级API更简洁地实现模型。
- 在PyTorch中，data模块提供了数据处理工具，nn模块定义了大量的神经网络层和常见损失函数。
- 我们可以通过_结尾的方法将参数替换，从而初始化参数。

### 3.3.9 练习

1. 如果将小批量的总损失替换为小批量损失的平均值，需要如何更改学习率？
    
    答. 将学习率`lr`除以批次大小`batch_size`

3. 如何访问线性回归的梯度？

    答. 使用`net[0].weight.grad`以及`net[0].bias.grad`来访问